In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

In [2]:
df = pd.read_csv('2010_Chronic_Conditions_PUF.csv')
df.head()

,BENE_SEX_IDENT_CD,BENE_AGE_CAT_CD,CC_ALZHDMTA,CC_CANCER,CC_CHF,CC_CHRNKIDN,CC_COPD,CC_DEPRESSN,CC_DIABETES,CC_ISCHMCHT,...,BENE_COUNT_PC_LT_12,AVE_MO_EN_PC_LT_12,BENE_COUNT_PC_EQ_12,BENE_COUNT_PD_LT_12,AVE_MO_EN_PD_LT_12,AVE_PDE_CST_PD_LT_12,AVE_PDE_PD_LT_12,BENE_COUNT_PD_EQ_12,AVE_PDE_CST_PD_EQ_12,AVE_PDE_PD_EQ_12
0,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,7230.0,49.877
1,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,42.0,5.952,2165.0,24.81,203.0,7163.0,73.404
2,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,4007.0,51.481
3,1,1,NaN,NaN,0,0,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.0,9021.0,75.732
4,1,1,NaN,NaN,0,0,NaN,NaN,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,5925.0,61.591


We will be looking at claims data from 2010 on Chronic Conditions. The data is PUF from the Center for Medicare and Medicaid Services. The link is provided below:

https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/BSAPUFS/Chronic_Conditions_PUF

First we do some investigating of the dataset itself.

In [3]:
df.shape

(22003, 55)

In [4]:
df.isnull().sum()/df.shape[0]*100

BENE_SEX_IDENT_CD         0.000000
BENE_AGE_CAT_CD           0.000000
CC_ALZHDMTA               3.490433
CC_CANCER                 3.490433
CC_CHF                    0.000000
CC_CHRNKIDN               0.000000
CC_COPD                   3.490433
CC_DEPRESSN               3.490433
CC_DIABETES               0.000000
CC_ISCHMCHT               0.000000
CC_OSTEOPRS               3.490433
CC_RA_OA                  0.000000
CC_STRKETIA               3.490433
CC_2_OR_MORE              0.000000
DUAL_STUS                 0.000000
BENE_COUNT_PA_LT_12      59.710040
AVE_MO_EN_PA_LT_12       59.710040
AVE_PA_PAY_PA_LT_12      60.228151
AVE_IP_PAY_PA_LT_12      64.323047
AVE_SNF_PAY_PA_LT_12     70.231332
AVE_OTH_PAY_PA_LT_12     70.231332
AVE_IP_ADM_PA_LT_12      64.323047
AVE_SNF_DAYS_PA_LT_12    70.231332
BENE_COUNT_PA_EQ_12       7.580784
AVE_PA_PAY_PA_EQ_12       8.489751
AVE_IP_PAY_PA_EQ_12      20.778985
AVE_SNF_PAY_PA_EQ_12     36.426851
AVE_OTH_PAY_PA_EQ_12     36.426851
AVE_IP_ADM_PA_EQ_12 

In [5]:
x = df['CC_CANCER'].isnull()
y= df['CC_ALZHDMTA'].isnull()

In [6]:
z=[]
for i in range(len(x)):
    if x[i]!=y[i]:
        z=z+[i]

z

[]

It seems the suppressed conditions are all in the same rows, so we removed the rows with suppressed data!

In [7]:
filtereddf = df.dropna(subset=['CC_CANCER'])

In [8]:
filtereddf.isnull().sum()/filtereddf.shape[0]*100

BENE_SEX_IDENT_CD         0.000000
BENE_AGE_CAT_CD           0.000000
CC_ALZHDMTA               0.000000
CC_CANCER                 0.000000
CC_CHF                    0.000000
CC_CHRNKIDN               0.000000
CC_COPD                   0.000000
CC_DEPRESSN               0.000000
CC_DIABETES               0.000000
CC_ISCHMCHT               0.000000
CC_OSTEOPRS               0.000000
CC_RA_OA                  0.000000
CC_STRKETIA               0.000000
CC_2_OR_MORE              0.000000
DUAL_STUS                 0.000000
BENE_COUNT_PA_LT_12      61.233812
AVE_MO_EN_PA_LT_12       61.233812
AVE_PA_PAY_PA_LT_12      61.770662
AVE_IP_PAY_PA_LT_12      65.947728
AVE_SNF_PAY_PA_LT_12     71.886037
AVE_OTH_PAY_PA_LT_12     71.886037
AVE_IP_ADM_PA_LT_12      65.947728
AVE_SNF_DAYS_PA_LT_12    71.886037
BENE_COUNT_PA_EQ_12       7.854956
AVE_PA_PAY_PA_EQ_12       8.796798
AVE_IP_PAY_PA_EQ_12      21.530492
AVE_SNF_PAY_PA_EQ_12     37.720744
AVE_OTH_PAY_PA_EQ_12     37.720744
AVE_IP_ADM_PA_EQ_12 

In [9]:
df=filtereddf
df.shape

(21235, 55)

That fixed the missing values in the conditions columns - and fortuitously we did not lose much! We note that the "Less than 12 month" columns all have a large amount of missing data, as do the Medicare plan C columns. We will quickly make a list of these "bad" indexes and remove them as part of our cleaning. The columns with a small percentage of missing values we will clean with SimpleImputer later!

In [10]:
bad=[]

for i in range(df.shape[1]):
    if df.iloc[:,i].isnull().sum()/df.shape[0]*100 >50:
        bad = bad + [i]

print(bad)

[15, 16, 17, 18, 19, 20, 21, 22, 30, 31, 32, 33, 34, 35, 36, 37, 45, 46, 47, 48, 49, 50, 51]


In [11]:
df = df.drop(df.columns[bad], axis=1)
df.head()

,BENE_SEX_IDENT_CD,BENE_AGE_CAT_CD,CC_ALZHDMTA,CC_CANCER,CC_CHF,CC_CHRNKIDN,CC_COPD,CC_DEPRESSN,CC_DIABETES,CC_ISCHMCHT,...,BENE_COUNT_PB_EQ_12,AVE_PB_PAY_PB_EQ_12,AVE_CA_PAY_PB_EQ_12,AVE_OP_PAY_PB_EQ_12,AVE_OTH_PAY_PB_EQ_12,AVE_CA_VST_PB_EQ_12,AVE_OP_VST_PB_EQ_12,BENE_COUNT_PD_EQ_12,AVE_PDE_CST_PD_EQ_12,AVE_PDE_PD_EQ_12
64,1,1,0.0,0.0,0,0,0.0,0.0,0,0,...,637117.0,890.0,473.0,278.0,139.0,1.948,1.127,516476.0,2480.0,29.415
65,1,1,0.0,0.0,0,0,0.0,0.0,0,0,...,581822.0,1688.0,829.0,564.0,295.0,2.957,2.786,753223.0,3479.0,28.795
66,1,1,0.0,0.0,0,0,0.0,0.0,0,0,...,4024.0,3064.0,1640.0,980.0,444.0,6.070,3.872,1685.0,2751.0,33.657
67,1,1,0.0,0.0,0,0,0.0,0.0,0,0,...,3865.0,5242.0,2629.0,1753.0,860.0,6.105,6.118,3963.0,3979.0,43.839
68,1,1,0.0,0.0,0,0,0.0,0.0,0,0,...,36156.0,3145.0,1978.0,851.0,316.0,7.099,3.109,16427.0,2876.0,34.996


In [12]:
df.dtypes

BENE_SEX_IDENT_CD          int64
BENE_AGE_CAT_CD            int64
CC_ALZHDMTA              float64
CC_CANCER                float64
CC_CHF                     int64
CC_CHRNKIDN                int64
CC_COPD                  float64
CC_DEPRESSN              float64
CC_DIABETES                int64
CC_ISCHMCHT                int64
CC_OSTEOPRS              float64
CC_RA_OA                   int64
CC_STRKETIA              float64
CC_2_OR_MORE               int64
DUAL_STUS                  int64
BENE_COUNT_PA_EQ_12      float64
AVE_PA_PAY_PA_EQ_12      float64
AVE_IP_PAY_PA_EQ_12      float64
AVE_SNF_PAY_PA_EQ_12     float64
AVE_OTH_PAY_PA_EQ_12     float64
AVE_IP_ADM_PA_EQ_12      float64
AVE_SNF_DAYS_PA_EQ_12    float64
BENE_COUNT_PB_EQ_12      float64
AVE_PB_PAY_PB_EQ_12      float64
AVE_CA_PAY_PB_EQ_12      float64
AVE_OP_PAY_PB_EQ_12      float64
AVE_OTH_PAY_PB_EQ_12     float64
AVE_CA_VST_PB_EQ_12      float64
AVE_OP_VST_PB_EQ_12      float64
BENE_COUNT_PD_EQ_12      float64
AVE_PDE_CS

In [13]:
df = df.astype('float64')
df2 = df

Just checking some different columns to get a sense of the data.

In [14]:
age_counts = df[['BENE_AGE_CAT_CD']].value_counts()
age_counts

BENE_AGE_CAT_CD
6.0                4305
5.0                3967
4.0                3826
3.0                3477
2.0                2974
1.0                2686
dtype: int64

In [15]:
sex_counts = df[['BENE_SEX_IDENT_CD']].value_counts()
sex_counts

BENE_SEX_IDENT_CD
2.0                  12008
1.0                   9227
dtype: int64

In [16]:
cancer_counts = df[["CC_CANCER"]].value_counts()
cancer_counts


CC_CANCER
0.0          15869
1.0           5366
dtype: int64

In [17]:
df[["CC_CANCER"]].isnull().sum()

CC_CANCER    0
dtype: int64

We now check for outliers! I am particularly interested in looking at drug prices (the last two columns pertaining to plan D).

In [18]:
df2.describe()

,BENE_SEX_IDENT_CD,BENE_AGE_CAT_CD,CC_ALZHDMTA,CC_CANCER,CC_CHF,CC_CHRNKIDN,CC_COPD,CC_DEPRESSN,CC_DIABETES,CC_ISCHMCHT,...,BENE_COUNT_PB_EQ_12,AVE_PB_PAY_PB_EQ_12,AVE_CA_PAY_PB_EQ_12,AVE_OP_PAY_PB_EQ_12,AVE_OTH_PAY_PB_EQ_12,AVE_CA_VST_PB_EQ_12,AVE_OP_VST_PB_EQ_12,BENE_COUNT_PD_EQ_12,AVE_PDE_CST_PD_EQ_12,AVE_PDE_PD_EQ_12
count,21235.000000,21235.000000,21235.000000,21235.000000,21235.000000,21235.000000,21235.000000,21235.000000,21235.000000,21235.000000,...,1.966300e+04,19663.000000,19663.000000,19517.000000,19517.000000,19663.000000,19517.000000,1.669900e+04,16699.000000,16699.000000
mean,1.565482,3.768966,0.414128,0.252696,0.472239,0.439604,0.422557,0.442194,0.483117,0.562703,...,1.531037e+03,11515.049891,6024.294970,3256.701747,2244.724906,12.434397,8.445115,1.550006e+03,5069.836457,70.628225
std,0.495705,1.675364,0.492582,0.434568,0.499240,0.496351,0.493978,0.496659,0.499727,0.496064,...,1.928658e+04,5540.731021,2936.794663,2159.437705,1449.120734,4.849661,2.805178,2.652404e+04,2110.611758,24.076963
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000e+01,890.000000,473.000000,254.000000,40.000000,1.209000,1.078000,3.000000e+01,1279.000000,17.259000
25%,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.900000e+01,7649.500000,3940.000000,1902.000000,1158.000000,8.945500,6.471000,5.500000e+01,3534.000000,52.124000
50%,2.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.330000e+02,10390.000000,5438.000000,2670.000000,1912.000000,12.413000,8.057000,1.180000e+02,4608.000000,67.065000
75%,2.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,4.130000e+02,14071.500000,7447.000000,3891.000000,3010.000000,15.785000,10.029000,3.430000e+02,6200.500000,86.422000
max,2.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.178926e+06,45752.000000,25764.000000,19355.000000,9938.000000,32.303000,26.912000,1.482514e+06,26641.000000,174.674000


In [19]:
sup = df2['AVE_PDE_CST_PD_EQ_12'].mean() +3*df2['AVE_PDE_CST_PD_EQ_12'].std()
inf = df2['AVE_PDE_CST_PD_EQ_12'].mean() -3*df2['AVE_PDE_CST_PD_EQ_12'].std()

print(sup,inf)

11401.67173259739 -1261.9988180516084


Since our lower bound is negative, we will only filter by the upper bound. While we should do a similar process for each column (via a for loop), my analysis will only be predicting drug prices.

In [20]:
df2 =  df2[df2['AVE_PDE_CST_PD_EQ_12'] < sup]

In [21]:
df2.shape

(16517, 32)

In [22]:
df2.isnull().sum()

BENE_SEX_IDENT_CD           0
BENE_AGE_CAT_CD             0
CC_ALZHDMTA                 0
CC_CANCER                   0
CC_CHF                      0
CC_CHRNKIDN                 0
CC_COPD                     0
CC_DEPRESSN                 0
CC_DIABETES                 0
CC_ISCHMCHT                 0
CC_OSTEOPRS                 0
CC_RA_OA                    0
CC_STRKETIA                 0
CC_2_OR_MORE                0
DUAL_STUS                   0
BENE_COUNT_PA_EQ_12       136
AVE_PA_PAY_PA_EQ_12       270
AVE_IP_PAY_PA_EQ_12      1865
AVE_SNF_PAY_PA_EQ_12     4569
AVE_OTH_PAY_PA_EQ_12     4569
AVE_IP_ADM_PA_EQ_12      1865
AVE_SNF_DAYS_PA_EQ_12    4569
BENE_COUNT_PB_EQ_12        59
AVE_PB_PAY_PB_EQ_12        59
AVE_CA_PAY_PB_EQ_12        59
AVE_OP_PAY_PB_EQ_12       148
AVE_OTH_PAY_PB_EQ_12      148
AVE_CA_VST_PB_EQ_12        59
AVE_OP_VST_PB_EQ_12       148
BENE_COUNT_PD_EQ_12         0
AVE_PDE_CST_PD_EQ_12        0
AVE_PDE_PD_EQ_12            0
dtype: int64

We will now clean up the missing (NaN) values. Since my target will be Plan D, we drop the missing values there to avoid bias. The remaining columns we will impute.

In [23]:
#But the above filtering already took care of that! But if we needed to drop more, we would do so as follows:

#df2 = df2.dropna(subset=['AVE_PDE_CST_PD_EQ_12'])

In [ ]:
imp_cost = SimpleImputer(missing_values=np.NaN, strategy='mean')
for i in range(15,32):
    df2.iloc[:,i] = imp_cost.fit_transform(df2.iloc[:,i].values.reshape(-1,1))[:,0]

df2.head()

In [25]:
df2.isnull().sum()/df2.shape[0]*100

BENE_SEX_IDENT_CD        0.0
BENE_AGE_CAT_CD          0.0
CC_ALZHDMTA              0.0
CC_CANCER                0.0
CC_CHF                   0.0
CC_CHRNKIDN              0.0
CC_COPD                  0.0
CC_DEPRESSN              0.0
CC_DIABETES              0.0
CC_ISCHMCHT              0.0
CC_OSTEOPRS              0.0
CC_RA_OA                 0.0
CC_STRKETIA              0.0
CC_2_OR_MORE             0.0
DUAL_STUS                0.0
BENE_COUNT_PA_EQ_12      0.0
AVE_PA_PAY_PA_EQ_12      0.0
AVE_IP_PAY_PA_EQ_12      0.0
AVE_SNF_PAY_PA_EQ_12     0.0
AVE_OTH_PAY_PA_EQ_12     0.0
AVE_IP_ADM_PA_EQ_12      0.0
AVE_SNF_DAYS_PA_EQ_12    0.0
BENE_COUNT_PB_EQ_12      0.0
AVE_PB_PAY_PB_EQ_12      0.0
AVE_CA_PAY_PB_EQ_12      0.0
AVE_OP_PAY_PB_EQ_12      0.0
AVE_OTH_PAY_PB_EQ_12     0.0
AVE_CA_VST_PB_EQ_12      0.0
AVE_OP_VST_PB_EQ_12      0.0
BENE_COUNT_PD_EQ_12      0.0
AVE_PDE_CST_PD_EQ_12     0.0
AVE_PDE_PD_EQ_12         0.0
dtype: float64

And now we have filled in the table! We will now make three new columns, one for the total cost for each plan.

In [ ]:
df2['Plan_A_Total_Cost'] = df2[df2.columns[17:20]].sum(axis=1)

In [ ]:
df2['Plan_B_Total_Cost'] = df2[df2.columns[24:27]].sum(axis=1)

In [ ]:
df2['Plan_D_Total_Cost'] = df2[df2.columns[30:31]].sum(axis=1)

In [29]:
df2.head()

,BENE_SEX_IDENT_CD,BENE_AGE_CAT_CD,CC_ALZHDMTA,CC_CANCER,CC_CHF,CC_CHRNKIDN,CC_COPD,CC_DEPRESSN,CC_DIABETES,CC_ISCHMCHT,...,AVE_OP_PAY_PB_EQ_12,AVE_OTH_PAY_PB_EQ_12,AVE_CA_VST_PB_EQ_12,AVE_OP_VST_PB_EQ_12,BENE_COUNT_PD_EQ_12,AVE_PDE_CST_PD_EQ_12,AVE_PDE_PD_EQ_12,Plan_A_Total_Cost,Plan_B_Total_Cost,Plan_D_Total_Cost
64,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,278.0,139.0,1.948,1.127,516476.0,2480.0,29.415,290.0,890.0,2480.0
65,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,564.0,295.0,2.957,2.786,753223.0,3479.0,28.795,913.0,1688.0,3479.0
66,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,980.0,444.0,6.070,3.872,1685.0,2751.0,33.657,3536.0,3064.0,2751.0
67,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1753.0,860.0,6.105,6.118,3963.0,3979.0,43.839,7164.0,5242.0,3979.0
68,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,851.0,316.0,7.099,3.109,16427.0,2876.0,34.996,1487.0,3145.0,2876.0


Now with a satisfactory table for analysis, we will save it as a data asset in our blob storage for easy access later.

In [30]:
df2.to_csv("cleaned_chronic_data.csv", index=False)

In [ ]:
#Connect to workspace (the actual values are hidden for security reasons)
subscription_id = "<MASKED>"
resource_group = "<MASKED>"
workspace = "<MASKED>"

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [ ]:
#Make table into a data asset for easy reference later

path = "azureml://<MASKED>/cleaned_chronic_data.csv"
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="Data for modeling",
    name="chronic-model-data",
    version="1",
)

ml_client.data.create_or_update(my_data)